In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os

In [2]:
df = pd.read_spss('data/CASPIANI.sav',convert_categoricals=True)  
df_iii = pd.read_spss('data/CASPIAN III.sav',convert_categoricals=True) 
df_iv = pd.read_spss('data/caspian4_.sav',convert_categoricals=True)
df_v = pd.read_spss('data/last-caspian-v.sav',convert_categoricals=True) 
def rename_features(df1, caspian_number):
    df2 = df1.copy()
    # Rename the features in the dataframe
    if caspian_number == 5:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 4:
        df2.drop('sex', axis=1, inplace=True)
        print(df2.columns)
        df2.rename(columns={'sex2': 'sex', 'weight_1': 'weight', 'height_2': 'height', 'University': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 3:
        df2.rename(columns={'weighte': 'weight', 'heighte': 'height', 'province': 'university', 'area':'region','p9':'schoolType'}, inplace=True)
    elif caspian_number == 1:
        df2.columns = df.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)
        

    return df2
df = rename_features(df,1)
df_iv=rename_features(df_iv,4)
df_iii = rename_features(df_iii,3)
df_v = rename_features(df_v,5)
df['sex'] = df['sex'].apply(lambda x: 'Girl' if x == 'Female' else 'Boy' if x == 'Male' else x)
df['region'] = df['region'].apply(lambda x: 'urban' if x == 'Urban' else 'rural' if x == 'Rural' else x)
df_iii['sex'] = df_iii['sex'].apply(lambda x: 'Boy' if x == 'female' else 'Girl' if x == 'male' else x)
df_iv['sex'] = df_iv['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)
df_iii['schoolType']=df_iii['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entefaei' else 'Unknown') 
df_iv['schoolType']=df_iv['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entef' else 'Unknown') 
df_v['schoolType']=df_v['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entefai' else 'Unknown') 
df_dict = {'caspian_I': df, 'caspian_III': df_iii, 'caspian_IV': df_iv, 'caspian_V': df_v}
def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["weight", "height", "sex", "age"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["height", "weight", "sex"])
        print(
            f"Number of records with NaN value in weight or height in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs


# Usage
processed_dfs = preprocess(df_dict)

Index(['id2', 'University', 'region', 'cluster', 'cross', 'birth_ye',
       'sample_c', 'a_1', 'a_2', 'a_3',
       ...
       'HeiMeter', 'bmi', 'CATBMI', 'CATBMIoutlieer', 'dicBMI',
       'socialsupport', 'PA1', 'PA2', 'KVDM', 'obes'],
      dtype='object', length=410)
Number of records with NaN value in weight or height in caspian_I: 11
Number of records with NaN value in weight or height in caspian_III: 0
Number of records with NaN value in weight or height in caspian_IV: 143
Number of records with NaN value in weight or height in caspian_V: 128


In [15]:
def growth_chart_without_smooth(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"

        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            # print(f"{percentile}th Percentile\nLength: {len(percentile_data)}\nData: {percentile_data.values}")
            # Ensure there’s data to plot
            if percentile_data.empty:
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Plot the raw percentile data without smoothing
            plt.plot(ages, percentile_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/raw'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_raw-{name}.png')
        print(f'{saving_folder}/{gender}-{feature}-age_raw-{name}.png')
        # plt.show()
        plt.close()
        print(f"Chart saved for {gender} in {name} with feature {feature}")

In [10]:
#bad bood
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def growth_chart_without_smooth(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        # Filter based on sex
        preprocess_df_sex = preprocess_df[preprocess_df['sex'] == sex]
        gender = "Boy" if sex == "Boy" else "Girl"

        if preprocess_df_sex.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        # Calculate percentiles for each age group and store sample data within each percentile range
        for age in preprocess_df_sex['age'].unique():
            age_group = preprocess_df_sex[preprocess_df_sex['age'] == age]
            # print(f"\nAge: {age}, Dataset: {name}, Feature: {feature}, Gender: {gender}")

            # Calculate percentile boundaries for the age group
            # percentile_thresholds = np.percentile(age_group[feature], percentiles)
            # for i, percentile in enumerate(percentiles):
            #     # If it's the lowest percentile, filter less than or equal to this threshold
            #     if i == 0:
            #         samples_in_percentile = age_group[age_group[feature] <= percentile_thresholds[i]]
            #     # For middle percentiles, filter between this and the previous threshold
            #     else:
            #         samples_in_percentile = age_group[(age_group[feature] > percentile_thresholds[i-1]) & 
            #                                           (age_group[feature] <= percentile_thresholds[i])]

                # Print the results
                # print(f"{percentile}th Percentile Range ({'≤' if i == 0 else '>'} {percentile_thresholds[i-1] if i > 0 else ''} "
                #       f"and ≤ {percentile_thresholds[i]}):")
                # print(f"Count: {len(samples_in_percentile)}")
                # print("Sample values:", samples_in_percentile[feature].values)

        # Visualization code remains the same
        ages = preprocess_df_sex['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = preprocess_df_sex.groupby('age')[feature].apply(lambda x: np.percentile(x, percentile))
            plt.plot(ages.unique(), percentile_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/raw'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_raw-{name}.png')
        plt.close()
        
        print(f"Chart saved for {gender} in {name} with feature {feature}")


In [ ]:
#bad bood
def growth_chart_without_smooth(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"

        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            # print(percentile,' \nlen:',len(percentile_data),'data is:',percentile_data)
            # Ensure there’s data to plot
            if percentile_data.empty:
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Plot the raw percentile data without smoothing
            plt.plot(ages, percentile_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/raw'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_raw-{name}.png')
        # plt.show()
        plt.close()
        print(f"Chart saved for {gender} in {name} with feature {feature}")
        # return percentile_results

        

In [17]:
# Define percentiles for each feature
percentile_settings = {
    'height': [3, 5, 10 , 25, 50, 75, 90, 95, 97],  # Percentiles for height
    'weight': [3, 5, 10 , 25, 50, 75, 90, 95, 97],  # Percentiles for weight
    'bmi1': [3, 5, 10 , 25, 50, 75, 85, 90, 95, 97]      # Different percentiles for BMI
}

# Define genders
genders = ['Boy', 'Girl']

# Fraction rate for LOESS smoothing
# window_size = 5
# Call the growth_chart function for each combination of feature and gender
for feature, percentiles in percentile_settings.items():
    for gender in genders:
        growth_chart_without_smooth(processed_dfs=processed_dfs, feature=feature, percentiles=percentiles, sex=gender)


/tmp/ipykernel_105708/3831077735.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


charts/raw/Boy-height-age_raw-caspian_I.png
Chart saved for Boy in caspian_I with feature height


/tmp/ipykernel_105708/3831077735.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


charts/raw/Boy-height-age_raw-caspian_III.png
Chart saved for Boy in caspian_III with feature height


/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_105708/3831077735.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


...

/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


charts/raw/Girl-bmi1-age_raw-caspian_III.png
Chart saved for Girl in caspian_III with feature bmi1


/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_105708/3831077735.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


charts/raw/Girl-bmi1-age_raw-caspian_IV.png
Chart saved for Girl in caspian_IV with feature bmi1
charts/raw/Girl-bmi1-age_raw-caspian_V.png
Chart saved for Girl in caspian_V with feature bmi1


/tmp/ipykernel_105708/3831077735.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def growth_chart_with_robust_percentiles(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_robust_percentiles(group):
            # Calculate Q1, Q3, and IQR
            Q1 = group[feature].quantile(0.25)
            Q3 = group[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define robust range limits
            lower_bound = Q1 - 1 * IQR
            upper_bound = Q3 + 1 * IQR
            
            # Filter data within the robust range
            robust_group = group[(group[feature] >= lower_bound) & (group[feature] <= upper_bound)]
            
            # Calculate percentiles on filtered data
            return np.percentile(robust_group[feature], percentiles) if not robust_group.empty else [np.nan] * len(percentiles)

        # Calculate robust percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_robust_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            # Ensure there’s data to plot
            if percentile_data.empty or percentile_data.isnull().all():
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Plot the robust percentile data
            plt.plot(ages, percentile_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/robust'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_robust-{name}.png')
        plt.close()
        print(f"Chart saved for {gender} in {name} with feature {feature}")


calculate robust percentiles

In [ ]:
percentiles=[3, 5, 10 , 25, 50, 75, 90, 95, 97]

In [ ]:
growth_chart_with_robust_percentiles(processed_dfs,'height',percentiles,'Boy')

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from statsmodels.nonparametric.smoothers_lowess import lowess

def growth_chart_with_robust_percentiles_and_loess(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_robust_percentiles(group):
            # Calculate Q1, Q3, and IQR
            Q1 = group[feature].quantile(0.25)
            Q3 = group[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define robust range limits
            lower_bound = Q1 - 2.5 * IQR
            upper_bound = Q3 + 2.5 * IQR
            
            # Filter data within the robust range
            robust_group = group[(group[feature] >= lower_bound) & (group[feature] <= upper_bound)]
            
            # Calculate percentiles on filtered data
            return np.percentile(robust_group[feature], percentiles) if not robust_group.empty else [np.nan] * len(percentiles)

        # Calculate robust percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_robust_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            # Ensure there’s data to plot
            if percentile_data.empty or percentile_data.isnull().all():
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue

            # Apply LOESS smoothing
            smoothed_data = lowess(percentile_data, ages, frac=0.5)  # Adjust frac for smoothing level
            
            # Plot the robust and smoothed percentile data
            plt.plot(smoothed_data[:, 0], smoothed_data[:, 1], label=f'{percentile}th Percentile (Smoothed)',
                     color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name} (LOESS Smoothed)')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/robust/smooth'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_robust_loess-{name}.png')
        plt.close()
        print(f"Smoothed chart saved for {gender} in {name} with feature {feature}")

growth_chart_with_robust_percentiles_and_loess(processed_dfs,'height',percentiles,'Boy')

In [56]:
def growth_chart_with_robust_percentiles_and_loess(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_robust_percentiles(group):
            # Calculate Q1, Q3, and IQR
            Q1 = group[feature].quantile(0.25)
            Q3 = group[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define robust range limits
            lower_bound = Q1 -1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Count total data points in the group
            total_count = len(group)

            # Filter data within the robust range
            robust_group = group[(group[feature] >= lower_bound) & (group[feature] <= upper_bound)]
            out_of_bounds = group[(group[feature] < lower_bound) | (group[feature] > upper_bound)]
            out_of_bounds_data = out_of_bounds[[feature, 'age', 'sex']] if not out_of_bounds.empty else pd.DataFrame()

            # Calculate percentiles on filtered data
            percentiles_result = np.percentile(robust_group[feature], percentiles) if not robust_group.empty else [np.nan] * len(percentiles)
            
            return total_count, *percentiles_result, out_of_bounds_data

        # Calculate robust percentiles
        results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_robust_percentiles(g), 
                                index=['total_count'] + [f'Percentile_{p}' for p in percentiles] + ['out_of_bounds_data'])
        )
        
        results.reset_index(inplace=True)
        
        if results.empty:
            print(f"No results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            results_gender = results[results['sex'] == 'Boy']
            gender = "Boy"
        else:
            results_gender = results[results['sex'] == 'Girl']
            gender = "Girl"

        if results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = results_gender['age']
        plt.figure(figsize=(12, 8))

        out_of_bounds_dict = {}  # Dictionary to collect out-of-bounds data by age and percentile

        for i, percentile in enumerate(percentiles):
            percentile_data = results_gender[f'Percentile_{percentile}']
            total_counts = results_gender['total_count']

            # Ensure there’s data to plot
            if percentile_data.empty or percentile_data.isnull().all():
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue

            # Apply LOESS smoothing
            smoothed_data = lowess(percentile_data, ages, frac=0.5)  # Adjust frac for smoothing level
            
            # Plot the robust and smoothed percentile data
            plt.plot(smoothed_data[:, 0], smoothed_data[:, 1], label=f'{percentile}th Percentile (Smoothed)',
                     color=plt.cm.plasma(i / len(percentiles)))

            # Collect out-of-bounds data specific to this percentile
            out_of_bounds_data = results_gender['out_of_bounds_data'].tolist()
            out_of_bounds_dict[percentile] = out_of_bounds_data

        # Output total count
        print(f"Total counts for {gender} in {name}:")
        for age, count in zip(ages, total_counts):
            print(f"  Age {age}: {count} data points")

        # Print out-of-bounds data specific to each age and percentile
        for percentile in percentiles:
            print(f"Out of bounds data for {gender} at {percentile}th percentile in {name}:")
            out_of_bounds_data = out_of_bounds_dict.get(percentile, [])
            if out_of_bounds_data:
                for ob in out_of_bounds_data:
                    print(ob)
                    # print(ob['age'])
            else:
                print("  No out-of-bounds data.")

        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name} (LOESS Smoothed)')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/robust/smooth'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_robust_loess-{name}.png')
        plt.close()
        print(f"Smoothed chart saved for {gender} in {name} with feature {feature}")



growth_chart_with_robust_percentiles_and_loess(processed_dfs,'height',percentiles,'Boy')


/tmp/ipykernel_31816/2720554975.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_31816/2720554975.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


Total counts for Boy in caspian_I:
  Age 7.0: 159 data points
  Age 8.0: 166 data points
  Age 9.0: 171 data points
  Age 10.0: 172 data points
  Age 11.0: 221 data points
  Age 12.0: 212 data points
  Age 13.0: 179 data points
  Age 14.0: 228 data points
  Age 15.0: 225 data points
  Age 16.0: 202 data points
  Age 17.0: 149 data points
  Age 18.0: 61 data points
Out of bounds data for Boy at 3th percentile in caspian_I:
Empty DataFrame
Columns: []
Index: []
      height  age  sex
199    101.0  8.0  Boy
990    150.0  8.0  Boy
1058   147.0  8.0  Boy
4651   112.0  8.0  Boy
4660   104.0  8.0  Boy
4671   111.0  8.0  Boy
      height  age  sex
3186   155.0  9.0  Boy
4654   113.0  9.0  Boy
4669   110.0  9.0  Boy
      height   age  sex
2966   158.0  10.0  Boy
      height   age  sex
766    168.0  11.0  Boy
1775   123.0  11.0  Boy
2257   112.0  11.0  Boy
3337   122.0  11.0  Boy
4032   109.0  11.0  Boy
      height   age  sex
1424   176.0  12.0  Boy
1516   178.0  12.0  Boy
4031   113.0  12.0 

/tmp/ipykernel_31816/2720554975.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_31816/2720554975.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


Total counts for Boy in caspian_III:
  Age 10.0: 63 data points
  Age 11.0: 166 data points
  Age 12.0: 412 data points
  Age 13.0: 350 data points
  Age 14.0: 304 data points
  Age 15.0: 327 data points
  Age 16.0: 295 data points
  Age 17.0: 387 data points
  Age 18.0: 497 data points
Out of bounds data for Boy at 3th percentile in caspian_III:
      height   age  sex
3081   160.0  10.0  Boy
      height   age  sex
1436   153.0  11.0  Boy
2379   164.0  11.0  Boy
2647   156.0  11.0  Boy
2675   154.0  11.0  Boy
2680   156.0  11.0  Boy
5291   155.0  11.0  Boy
      height   age  sex
2726   106.0  12.0  Boy
2969   164.0  12.0  Boy
3030   113.0  12.0  Boy
3034   110.0  12.0  Boy
3248   163.0  12.0  Boy
4210   164.0  12.0  Boy
4283   170.0  12.0  Boy
4429   197.0  12.0  Boy
      height   age  sex
1098   167.0  13.0  Boy
2360   121.0  13.0  Boy
2437   174.0  13.0  Boy
4423   116.0  13.0  Boy
4576   104.0  13.0  Boy
4605   167.0  13.0  Boy
4911   120.0  13.0  Boy
      height   age  sex
122

/tmp/ipykernel_31816/2720554975.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


Total counts for Boy in caspian_IV:
  Age 7.0: 350 data points
  Age 8.0: 549 data points
  Age 9.0: 637 data points
  Age 10.0: 744 data points
  Age 11.0: 659 data points
  Age 12.0: 666 data points
  Age 13.0: 778 data points
  Age 14.0: 641 data points
  Age 15.0: 453 data points
  Age 16.0: 632 data points
  Age 17.0: 585 data points
  Age 18.0: 326 data points
Out of bounds data for Boy at 3th percentile in caspian_IV:
           height  age  sex
3627   148.000000  7.0  Boy
3874   145.000000  7.0  Boy
3946   145.000000  7.0  Boy
5202   144.500000  7.0  Boy
5245   156.000000  7.0  Boy
5568   150.000000  7.0  Boy
6475   149.000000  7.0  Boy
7901   178.000000  7.0  Boy
9605   102.000000  7.0  Boy
10393  150.000000  7.0  Boy
10679  158.000000  7.0  Boy
11296  161.000000  7.0  Boy
11523  102.000000  7.0  Boy
11959  155.600006  7.0  Boy
12056  102.000000  7.0  Boy
12897  150.000000  7.0  Boy
12938  176.500000  7.0  Boy
           height  age  sex
2480   162.000000  8.0  Boy
2923   147.

/tmp/ipykernel_31816/2720554975.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_31816/2720554975.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


Total counts for Boy in caspian_V:
  Age 7.0: 359 data points
  Age 8.0: 561 data points
  Age 9.0: 655 data points
  Age 10.0: 768 data points
  Age 11.0: 672 data points
  Age 12.0: 684 data points
  Age 13.0: 787 data points
  Age 14.0: 658 data points
  Age 15.0: 463 data points
  Age 16.0: 646 data points
  Age 17.0: 591 data points
  Age 18.0: 329 data points
Out of bounds data for Boy at 3th percentile in caspian_V:
           height  age  sex
421    178.000000  7.0  Boy
460    148.000000  7.0  Boy
1400   102.000000  7.0  Boy
1859   145.000000  7.0  Boy
1929   144.500000  7.0  Boy
4806   161.000000  7.0  Boy
5626   102.000000  7.0  Boy
6113   150.000000  7.0  Boy
6115   156.000000  7.0  Boy
6116   158.000000  7.0  Boy
6117   149.000000  7.0  Boy
7143    91.000000  7.0  Boy
7146    93.000000  7.0  Boy
7165   154.000000  7.0  Boy
7167   150.000000  7.0  Boy
7392   102.000000  7.0  Boy
8557   150.000000  7.0  Boy
12483  176.500000  7.0  Boy
12773  155.600006  7.0  Boy
13340  150.00

In [ ]:
import pandas as pd

def growth_chart_with_robust_percentiles_and_loess(processed_dfs, feature, percentiles, sex):
    all_out_of_bounds_data = []  # List to accumulate out-of-bounds data across all groups
    
    for name, preprocess_df in processed_dfs.items():
        def calculate_robust_percentiles(group):
            # Calculate Q1, Q3, and IQR
            Q1 = group[feature].quantile(0.25)
            Q3 = group[feature].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define robust range limits
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Count total data points in the group
            total_count = len(group)

            # Filter data within the robust range
            robust_group = group[(group[feature] >= lower_bound) & (group[feature] <= upper_bound)]
            out_of_bounds = group[(group[feature] < lower_bound) | (group[feature] > upper_bound)]
            out_of_bounds_data = out_of_bounds[['age', 'sex', feature]] if not out_of_bounds.empty else pd.DataFrame()
            
            # Append out-of-bounds data to the list
            if not out_of_bounds_data.empty:
                all_out_of_bounds_data.append(out_of_bounds_data)
            
            # Calculate percentiles on filtered data
            percentiles_result = np.percentile(robust_group[feature], percentiles) if not robust_group.empty else [np.nan] * len(percentiles)
            
            return total_count, *percentiles_result

        # Calculate robust percentiles
        results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_robust_percentiles(g), 
                                index=['total_count'] + [f'Percentile_{p}' for p in percentiles])
        )
        
        results.reset_index(inplace=True)
        
        # Process other parts of the code, such as plotting (omitted here for brevity)

    # Save all out-of-bounds data to a CSV after processing all groups
    if all_out_of_bounds_data:
        combined_out_of_bounds = pd.concat(all_out_of_bounds_data, ignore_index=True)
        combined_out_of_bounds.to_csv('out_of_bounds_data.csv', index=False)
        print("Out-of-bounds data saved to out_of_bounds_data.csv")
    else:
        print("No out-of-bounds data to save.")

growth_chart_with_robust_percentiles_and_loess(processed_dfs,'height',percentiles,'Boy')


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

def growth_chart_with_smoothing(processed_dfs, feature, percentiles, sex, smooth=False, smooth_type='moving_average', window_size=3):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']

            # Check if there's data to plot
            if percentile_data.empty:
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Apply smoothing if enabled
            if smooth:
                if smooth_type == 'moving_average':
                    # Apply moving average smoothing
                    smoothed_data = percentile_data.rolling(window=window_size, min_periods=1).mean()
                elif smooth_type == 'spline':
                    # Apply spline interpolation for smoothing
                    spline = make_interp_spline(ages, percentile_data, k=3)  # cubic spline
                    smoothed_data = spline(np.linspace(ages.min(), ages.max(), 200))
                    ages_smoothed = np.linspace(ages.min(), ages.max(), 200)
                else:
                    raise ValueError("Invalid smooth_type. Choose 'moving_average' or 'spline'.")
            else:
                smoothed_data = percentile_data
                ages_smoothed = ages

            # Plot the data
            plt.plot(ages_smoothed if smooth and smooth_type == 'spline' else ages, smoothed_data,
                     label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name} {"(Smoothed)" if smooth else ""}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/{"smooth" if smooth else "-raw"}'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_{smooth_type if smooth else "raw"}-{name}.png')
        plt.show()
        # plt.close()

        print(f"Chart saved for {gender} in {name} with feature {feature} {'(Smoothed)' if smooth else ''}")
        return percentile_results
# Example usage:
growth_chart_with_smoothing(processed_dfs, 'height', percentiles, 'Boy', smooth=True, smooth_type='spline', window_size=5)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def logarithmic_model(x, a, b):
    return a + b * np.log(x)

def growth_chart_logarithmic_fit(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']

            # Fit a logarithmic model to the data
            try:
                # Fit the logarithmic model only if age values are positive
                popt, _ = curve_fit(logarithmic_model, ages[ages > 0], percentile_data[ages > 0])
                fitted_data = logarithmic_model(ages, *popt)
            except RuntimeError as e:
                print(f"Could not fit logarithmic model for {percentile}th percentile in {name}: {e}")
                continue
            
            # Plot the fitted curve
            plt.plot(ages, fitted_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name} (Logarithmic Fit)')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/logarithmic-fit'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_logarithmic_fit-{name}.png')
        plt.show()
        # plt.close()

        print(f"Logarithmic fit chart saved for {gender} in {name} with feature {feature}")

# Example usage:
growth_chart_logarithmic_fit(processed_dfs, 'height', percentiles, 'Boy')


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Logistic growth function
def logistic_model(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))

def growth_chart_logistic_fit(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']

            # Fit a logistic model to the data
            try:
                # Initial guess for parameters L, k, and x0
                L_initial = percentile_data.max()  # Maximum height for this percentile
                k_initial = 0.1  # Initial guess for growth rate
                x0_initial = ages.median()  # Initial guess for inflection point

                popt, _ = curve_fit(logistic_model, ages, percentile_data, p0=[L_initial, k_initial, x0_initial])
                fitted_data = logistic_model(ages, *popt)
            except RuntimeError as e:
                print(f"Could not fit logistic model for {percentile}th percentile in {name}: {e}")
                continue
            
            # Plot the fitted curve
            plt.plot(ages, fitted_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name} (Logistic Fit)')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/logistic-fit'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_logistic_fit-{name}.png')
        plt.show()
        # plt.close()

        print(f"Logistic fit chart saved for {gender} in {name} with feature {feature}")

# Example usage:
growth_chart_logistic_fit(processed_dfs, 'height', percentiles, 'Boy')
